In [1]:
/**
 * Calculate the mean and variance of the stochastic integral:
 *
 *  E( ∫ B2(t) dB1(t) ) for [0, 1], where B1, B2 are two different Brownian motions.
 *
 * We calculate the statistics by generating filtrations from B1.
 * For each filtration, we evaluate the integral w.r.t. that filtration to obtain one data point.
 *
 * @author Haksun Li
 */

%use s2

// the number of discretizations
val N_T = 1000

// the number of simulations
val N_sims = 100

// the time of the first point
val t_0 = 0.0

// the time of the last point
val t_1 = 1.0

// the seed for random number generators
val seed = 1234567890L

val T: TimeGrid = EvenlySpacedGrid(t_0, t_1, N_T) // discretization
val sde: DiscreteSDE = BMSDE() // brownian motion stochastic differential equation
val x0 = 0.0
val B: RandomRealizationGenerator = RandomRealizationOfRandomProcess(sde, T, x0) // an RNG to generate random paths
B.seed(seed)

val mu = Mean()
val `var` = Variance()

for (i in 0 until N_sims) {
    // generate a Brownian filtration
    val Ft = Filtration(B.nextRealization())

    // a new B2 is created for each iteration, hence a different FT2
    val B2: FiltrationFunction = object : FiltrationFunction() {
        // generate a different Brownian filtration
        private val Ft2: Filtration = Filtration(B.nextRealization())
        override fun evaluate(t: Int): Double {
            return Ft2.B(t)
        }
    }
    
    val I: Double = IntegralDB(B2).value(Ft) // evaluate the integral w.r.t. to the particular realization
    mu.addData(I) // collect the statistics
    `var`.addData(I) // collect the statistics
}

println("the mean is %f%n".format(mu.value()))
println("the variance is %f%n".format(`var`.value()))

the mean is -0.152701

the variance is 0.667297

